In [ ]:
%matplotlib inline
%load_ext tensorboard

import matplotlib.pyplot as plt
import nengo
import numpy as np
import tensorflow as tf

import datetime, os
from itertools import islice
from IPython.display import clear_output
import pytry
import pickle
import random

import nengo_dl

# Deep Learning Nengo Network

Tensorflow version 2.2.0 <br>
Nengo dl version 3.0.0

In [ ]:
class FeedForwardTrain(pytry.Trial):
    def params(self):
        self.param('run_n', run_n=1)
    
    def evaluate(self, param):       
        #LOAD/SET PARAMETERS
        run_n=param.run_n      
        learn_rate=0.000001 #learning rate
        n_epoch=200 #number of training epochs
        train_rat=0.7 #ratio of training to testing data
        
        print(run_n) #print run number
        np.random.seed() #set seed to random value
        
#################################################################
        #Set and create data directories
        data_dir="./NengoDL_LMU/" #directory to save data to
        
        if run_n == 0:
            os.mkdir(data_dir)
            os.mkdir('.\\NengoDL_LMU_logs\\')
            
        logdir = os.path.join(".\\NengoDL_LMU_logs\\"+str(run_n)) #directory to save logs to for tensorboard
        
        #load data
        highengdata = np.load('high_lmu4_theta3.npy',allow_pickle=True) #np.load('higheng.npy', allow_pickle=True)
        lowengdata = np.load('low_lmu4_theta3.npy',allow_pickle=True) #np.load('loweng.npy', allow_pickle=True)
##################################################################

        high=list(highengdata[:])
        low=list(lowengdata[:])

        #shuffle clips
        np.random.shuffle(high)
        np.random.shuffle(low)

        #get training clips and convert to list of frames
        high_train = high[:(int(len(low)*train_rat))]
        high_train=list(np.vstack(high_train))
        low_train = low[:(int(len(low)*train_rat))]
        low_train=list(np.vstack(low_train))

        #shuffle frames
        np.random.shuffle(high_train)
        np.random.shuffle(low_train)

        #Make each training set same size (noplay is shortest)
        high_train = high_train[:(int(len(low_train)))]
        low_train = low_train[:(int(len(low_train)))]

        #Create 10% validation dataset
        goal_val = goal[(int(len(noplay)*0.9)):(int(len(noplay)*1))]
        noplay_val = noplay[(int(len(noplay)*0.9)):(int(len(noplay)*1))]
        
        #convert to list of frames
        goal_val=list(np.vstack(goal_val))
        noplay_val=list(np.vstack(noplay_val))

        #shuffle frames
        np.random.shuffle(goal_val)
        np.random.shuffle(noplay_val)
        
        #create test sets
        #get 30% of clips for testing
        goal_test = goal[(int(len(noplay)*train_rat)):(int(len(noplay)*0.9))]
        noplay_test = noplay[(int(len(noplay)*train_rat)):(int(len(noplay)*0.9))]

        #shuffle frames
        np.random.shuffle(goal_test)
        np.random.shuffle(noplay_test)

        #GENERATE INPUT MATRICES
        #Concatenate goal and noplay sets together to create a single array for training, test and validation separately
        all_train = np.vstack(np.concatenate((high_train, low_train)))
        all_test = np.concatenate((goal_test, noplay_test)) 
        all_val = np.concatenate((goal_val, noplay_val))

        #create the target data for training
        target_train = np.zeros((all_train.shape[0],2))
        n_goal = len(goal_train)
        target_train[:n_goal,0] = 1 #target for goal = [1,0]
        target_train[n_goal:,1] = 1 #target for noplay = [0,1]
        
        #and for validation
        target_val = np.zeros((all_val.shape[0],2))
        n_goal = len(goal_val)
        target_val[:n_goal,0] = 1
        target_val[n_goal:,1] = 1
        
        #SAVE TRAINING AND TESTING DATA
        pickle_filename = (data_dir+"/%s_training_data.pkl") % str(run_n) #(data_dir+"/%s_training_data"+str(test_param)+".pkl") % run_n
        with open(pickle_filename, 'wb') as file:
            pickle.dump(all_train, file)

        pickle_filename = (data_dir+"/%s_testing_data.pkl") % str(run_n) #(data_dir+"/%s_testing_data"+str(test_param)+".pkl") % run_n
        with open(pickle_filename, 'wb') as file:
            pickle.dump(all_test, file)

        #BUILD MODEL (input layer -> hidden layer -> output layer)
        N = 200
        seed = 1
        N_dims = len(all_train[0]) 

        model = nengo.Network(seed=seed)
        with model:    
            input = nengo.Node(np.zeros(N_dims))
            hidden1 = nengo.Ensemble(n_neurons=N, dimensions=N_dims, radius=np.sqrt(N_dims), 
                                     neuron_type=nengo.RectifiedLinear())
            nengo.Connection(input, hidden1, synapse=None)
            output = nengo.Node(None, size_in=2)
            nengo.Connection(hidden1, output, eval_points=all_train, function=target_train, 
                             scale_eval_points=False, synapse=None)

            p_output = nengo.Probe(output)
        
        #TRAINING DATA AND PARAMETERS
        minibatch_size = 1000       # this can be adjusted to speed up training
                                    # Note: with larger minibatch_size, you may need to reduce this
                                    #(if training is making things worse, then you need to reduce this!)
                                    # number of iterations through the training data to perform

        batches = int(np.ceil(len(all_train)/minibatch_size)) #number of training batches needed (length of data / minibatch size)
        val_batches = int(np.ceil(len(all_val)/minibatch_size)) #number of validation batches needed

        training_data_input = np.array(all_train, copy=True)
        order = np.arange(len(training_data_input))
        np.random.shuffle(order)
        training_data_input_rand = training_data_input[order]
        training_data_input_rand.resize(minibatch_size,batches,N_dims)
        training_data_input.resize(minibatch_size,batches,N_dims)

        training_data_output = np.array(target_train, copy=True)
        training_data_output_rand = training_data_output[order]
        training_data_output.resize(minibatch_size,batches,2)
        training_data_output_rand.resize(minibatch_size,batches,2)
        
        val_input = np.array(all_val, copy=True)
        order = np.arange(len(val_input))
        np.random.shuffle(order)
        val_input_rand = val_input[order]
        val_input_rand.resize(minibatch_size,val_batches,N_dims)
        val_input.resize(minibatch_size,val_batches,N_dims)
        
        val_output = np.array(target_val, copy=True)
        val_output_rand = val_output[order]
        val_output.resize(minibatch_size,val_batches,2)
        val_output_rand.resize(minibatch_size,val_batches,2)
        
        #set learning objective (defines loss)
        def objective(outputs, targets): 
            return tf.compat.v1.nn.softmax_cross_entropy_with_logits_v2(
                logits=outputs, labels=targets)  
        
        #TRAINING
        with nengo_dl.Simulator(
                model, minibatch_size=minibatch_size) as sim:
            sim.compile(optimizer=tf.optimizers.Adam(learn_rate),
                loss={p_output: objective})
            sim.fit(training_data_input_rand, training_data_output_rand, epochs=n_epoch,
                   callbacks=[tf.keras.callbacks.TensorBoard(log_dir=logdir)],
                   validation_data=(val_input_rand, val_output_rand))
            sim.save_params(data_dir+"/%s_trained.data" % str(run_n)) #(data_dir+"/%s"+str(test_param)+"_trained.data") % run_n #('./trained.data') 
            
        #Run model with training data, using trained weights
        input.output = nengo.processes.PresentInput(all_train, presentation_time=0.001)
        with nengo_dl.Simulator(model, minibatch_size=1) as sim:
            sim.load_params(data_dir+"/%s_trained.data" % str(run_n)) #(data_dir+"/%s"+str(test_param)+"_trained.data") % run_n
            sim.run(len(all_train)*0.001)
            
        #Save data
        pickle_filename = (data_dir+"/%s_train_target.pkl") % str(run_n) #(data_dir+"/%s_train_target"+str(test_param)+".pkl") % run_n
        with open(pickle_filename, 'wb') as file:
            pickle.dump(target_train, file)
        
        pickle_filename = (data_dir+"/%s_train_trange.pkl") % str(run_n) #(data_dir+"/%s_train_trange"+str(test_param)+".pkl") % run_n
        with open(pickle_filename, 'wb') as file:
            pickle.dump(sim.trange(), file)
            
        pickle_filename = (data_dir+"/%s_train_out.pkl") % str(run_n) #(data_dir+"/%s_train_out"+str(test_param)+".pkl") % run_n
        with open(pickle_filename, 'wb') as file:
            pickle.dump(sim.data[p_output][0], file)    
        
        test_out=[]
        target_test=[]
        for j in range(len(all_test)):
            if j<int(len(goal_test)):
                target = np.zeros((all_test[j].shape[0],2))
                target[:,0] = 1
            else:
                target = np.zeros((all_test[j].shape[0],2))
                target[:,1] = 1
            input.output = nengo.processes.PresentInput(all_test[j], presentation_time=0.001)
            with nengo_dl.Simulator(model, minibatch_size=1) as sim:
                sim.load_params(data_dir+"/%s_trained.data" % str(run_n)) #(data_dir+"/%s"+str(test_param)+"_trained.data") % run_n
                sim.run(len(all_test[j])*0.001)
            test_out.append(sim.data[p_output][0])
            target_test.append(target)
            
        #Save data
        pickle_filename = (data_dir+"/%s_test_target.pkl") % str(run_n) #(data_dir+"/%s_test_target"+str(test_param)+".pkl") % run_n
        with open(pickle_filename, 'wb') as file:
            pickle.dump(target_test, file)
        
        pickle_filename = (data_dir+"/%s_test_trange.pkl") % str(run_n) #(data_dir+"/%s_test_trange"+str(test_param)+".pkl") % run_n
        with open(pickle_filename, 'wb') as file:
            pickle.dump(sim.trange(), file)
            
        pickle_filename = (data_dir+"/%s_test_out.pkl") % str(run_n) #(data_dir+"/%s_test_out"+str(test_param)+".pkl") % run_n
        with open(pickle_filename, 'wb') as file:
            pickle.dump(test_out, file)


In [ ]:
for run in range(20):
    FeedForwardTrain().run(run_n=run)

To view logs in tensorboard use following command in command prompt: <br>
    tensorboard --logdir={logs_base_dir} --host=localhost <br>
<br>
Note: change {logs_base_dir} to the name of the folder where logs are being saved. <br>
Note: make sure you have navigated to the directory containing this script in the command prompt before running the above command.